# 第7章: チャットアプリケーションの構築
## OpenAI API クイックスタート

このノートブックは、[Azure OpenAI サンプルリポジトリ](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst)からのものを元にしています。このリポジトリには、[Azure OpenAI](notebook-azure-openai.ipynb) サービスにアクセスするノートブックが含まれています。

Python の OpenAI API は、いくつかの変更を加えることで Azure OpenAI モデルでも利用できます。違いについて詳しくは、こちらをご覧ください: [Python で OpenAI と Azure OpenAI のエンドポイントを切り替える方法](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)


# 概要  
「大規模言語モデルは、テキストからテキストへの写像を行う関数です。入力されたテキストに対して、大規模言語モデルは次に続くテキストを予測しようとします」(1)。この「クイックスタート」ノートブックでは、LLMの基本的な概念、AMLを始めるために必要な主要パッケージ、プロンプト設計の簡単な紹介、そしてさまざまなユースケースの短い例をいくつか紹介します。


## 目次

[概要](../../../../07-building-chat-applications/python)  
[OpenAI サービスの使い方](../../../../07-building-chat-applications/python)  
[1. OpenAI サービスの作成](../../../../07-building-chat-applications/python)  
[2. インストール](../../../../07-building-chat-applications/python)  
[3. 認証情報](../../../../07-building-chat-applications/python)  

[ユースケース](../../../../07-building-chat-applications/python)  
[1. テキストの要約](../../../../07-building-chat-applications/python)  
[2. テキストの分類](../../../../07-building-chat-applications/python)  
[3. 新しい商品名の生成](../../../../07-building-chat-applications/python)  
[4. 分類器のファインチューニング](../../../../07-building-chat-applications/python)  

[参考資料](../../../../07-building-chat-applications/python)


### 最初のプロンプトを作成しよう  
この短い演習では、OpenAIモデルに「要約」というシンプルなタスクのプロンプトを送信する基本的な方法を紹介します。

**手順**:  
1. Python環境にOpenAIライブラリをインストールする  
2. 標準的なヘルパーライブラリを読み込み、作成したOpenAIサービス用のセキュリティ認証情報を設定する  
3. タスクに使うモデルを選択する  
4. モデル用のシンプルなプロンプトを作成する  
5. モデルAPIにリクエストを送信する


In [ ]:
%pip install openai python-dotenv

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )


### 3. 適切なモデルを見つける

GPT-3.5-turbo や GPT-4 モデルは、自然言語を理解し生成することができます。


In [ ]:
# Select the General Purpose curie model for text
model = "gpt-3.5-turbo"

## 4. プロンプト設計

「大規模言語モデルの魔法は、膨大な量のテキストで予測誤差を最小化するように訓練されることで、モデルがこれらの予測に役立つさまざまな概念を学習する点にあります。たとえば、モデルは次のような概念を学びます」(1):

* スペルの仕方
* 文法の仕組み
* 言い換えの方法
* 質問への答え方
* 会話の進め方
* 多言語での文章の書き方
* コードの書き方
* など

#### 大規模言語モデルの制御方法
「大規模言語モデルへの入力の中で、圧倒的に影響力が大きいのはテキストプロンプトです」(1)。

大規模言語モデルは、いくつかの方法で出力を促すことができます：

インストラクション: モデルにやってほしいことを伝える  
コンプリーション: 欲しい内容の冒頭を与えて続きを生成させる  
デモンストレーション: モデルにやってほしいことを例で示す（以下のいずれかで）  
プロンプト内にいくつかの例を入れる  
ファインチューニング用の学習データセットに何百、何千もの例を入れる

#### プロンプト作成の基本的なガイドラインは3つあります：

**見せて、伝えること。** 指示や例、またはその両方を使って、何をしてほしいのかを明確に伝えましょう。たとえば、リストをアルファベット順に並べ替えてほしい場合や、段落を感情で分類してほしい場合は、それが目的であることを示してください。

**質の高いデータを用意すること。** 分類器を作りたい場合や、モデルにパターンを守らせたい場合は、十分な数の例を用意しましょう。例文は必ず見直してください。モデルは基本的なスペルミスを見抜いて返答してくれることも多いですが、意図的なものだと判断して返答に影響することもあります。

**設定を確認すること。** temperatureやtop_pの設定は、モデルの応答がどれだけ決定的（ランダム性が少ない）かを制御します。正解が一つしかないような問いの場合は、これらの値を低く設定しましょう。多様な応答が欲しい場合は高めに設定します。よくある間違いは、これらの設定を「賢さ」や「創造性」のコントロールだと勘違いすることです。

出典: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## テキストの要約  
#### チャレンジ  
テキストの最後に「tl;dr:」を追加して要約します。モデルが追加の指示なしで様々なタスクをこなせることに注目してください。tl;dr以外にも、より説明的なプロンプトを使ってモデルの動作を変えたり、要約の内容をカスタマイズすることもできます(3)。  

最近の研究では、大規模なテキストコーパスで事前学習し、その後特定のタスクでファインチューニングすることで、多くのNLPタスクやベンチマークで大きな成果が得られることが示されています。通常、この手法はアーキテクチャがタスク非依存ですが、依然として数千から数万のタスク固有のファインチューニング用データセットが必要です。一方で、人間は少数の例や簡単な指示だけで新しい言語タスクをこなすことができますが、現在のNLPシステムはこの点でまだ大きく苦戦しています。本稿では、言語モデルの規模を拡大することで、タスク非依存かつ少数例でのパフォーマンスが大きく向上し、従来の最先端ファインチューニング手法と競合するレベルに達することもあることを示します。



Tl;dr


# いくつかのユースケースのための練習問題  
1. テキストの要約  
2. テキストの分類  
3. 新しい商品名の生成


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## テキストの分類  
#### チャレンジ  
推論時に指定されたカテゴリにアイテムを分類します。次の例では、プロンプト内でカテゴリと分類するテキストの両方を指定しています（*playground_reference）。

顧客からの問い合わせ: こんにちは、ノートパソコンのキーボードのキーの一つが最近壊れてしまい、交換が必要です。

分類されたカテゴリ:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## 新しい商品名を考えよう
#### チャレンジ
例となる単語から商品名を作成します。このプロンプトでは、名前を考える商品の情報も含めています。また、どのようなパターンで名前を作ってほしいかを示すため、似た例も用意しています。さらに、ランダム性や独創的な回答を増やすために、temperature値を高めに設定しています。

商品説明: 家庭用ミルクセーキメーカー  
シードワード: 速い、健康、コンパクト  
商品名例: HomeShaker、Fit Shaker、QuickShake、Shake Maker

商品説明: どんな足のサイズにも合う靴  
シードワード: 適応、フィット、オムニフィット


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# 参考文献  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studioの例](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [GPT-3を微調整してテキストを分類するためのベストプラクティス](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# 詳しいサポートについて  
[OpenAI コマーシャライゼーションチーム](AzureOpenAITeam@microsoft.com)


# 貢献者
* Louis Li



---

**免責事項**:  
本書類は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性には努めておりますが、自動翻訳には誤りや不正確な表現が含まれる場合があります。原文（元の言語の文書）が正式な情報源と見なされるべきです。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用により生じたいかなる誤解や誤訳についても、当方は一切の責任を負いません。
